In [ ]:
# Define the path to the results directory
directory = '/data2/fabricehc/sandcastles/distinguisher/results'
# Python change import directory to include parent directory
import sys
import os
sys.path.insert(0, '..')
from IPython.display import display, HTML

def show(df):
    # Display the DataFrame in a scrollable container
    display(HTML('<div style="max-height: 300px; overflow-y: scroll;">{}</div>'.format(
        df.to_html(index=False)
    )))

In [33]:
import pandas as pd
from matplotlib import pyplot as plt
from distinguisher.utils import prompt_to_entropy, extract_unique_column_value

def combine_files_by_base_name(prefix, distinguisher):
    """
    Combines files in a directory based on their base name, excluding certain files.

    Parameters:
        directory (str): Path to the directory containing files to combine.

    Returns:
        dict: A dictionary where keys are base names and values are combined DataFrames.
    """
    file_groups = {}

    for filename in os.listdir(directory):
        if filename.split('_')[0] != prefix or filename.split('_')[-1].split('.')[0] != distinguisher:
            continue
        base_name = '_'.join(filename.split('_part')[0].split('_')[:-1])
        if base_name not in file_groups:
            file_groups[base_name] = []
        file_groups[base_name].append(filename)

    combined_dfs = {}
    for base_name, files in file_groups.items():
        combined_df = pd.DataFrame()

        for filename in sorted(files):  # Ensure files are processed in order of parts
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)

        combined_dfs[base_name] = combined_df

    return combined_dfs

# Helper function to separate attacks
def separate_attacks(df, length=10000):
    attacks = []
    current_attack = []
    
    for idx, row in df.iterrows():
        # Distinguishing experiments based on alternating 'Origin' values
        if idx > 0 and row['Origin'] != df.loc[idx - 1, 'Origin']:
            attacks.append(pd.DataFrame(current_attack))
            current_attack = []        

        current_attack.append(row)
    
    # Append the last attack
    if current_attack:
        attacks.append(pd.DataFrame(current_attack))
    
    return attacks

def add_entropy_column(dfs):
    """
    Takes a list of DataFrame and adds an 'entropy' column to each DataFrame

    Parameters:
        dfs (list of pd.DataFrame): List of DataFrames.

    Returns:
        list of pd.DataFrame: The updated list of DataFrames with the added 'entropy' column.
    """
    for df in dfs:
        df['entropy'] = prompt_to_entropy(extract_unique_column_value(df, 'prompt'))

    return dfs

def process_and_add_entropy(file_path, separate_attacks, add_entropy_column):
    """
    Streamlines the processing of a CSV file: reads it, extracts domain from filename,
    adds a domain column, separates attacks, and adds an entropy column.

    Parameters:
        file_path (str): Path to the CSV file.
        separate_attacks (function): Function to separate attacks from the DataFrame.
        add_entropy_column (function): Function to add an entropy column to the list of DataFrames.

    Returns:
        list of pd.DataFrame: List of processed DataFrames with entropy column added.
    """
    domain = os.path.basename(file_path).split('_')[-2]
    df = pd.read_csv(file_path)
    df['domain'] = domain
    attacks = separate_attacks(df)
    return add_entropy_column(attacks)

def check_unique_values(df, columns):
    """
    Checks whether specified columns in a DataFrame have a single unique value and extracts this value.

    Parameters:
        df (pd.DataFrame): The DataFrame to check.
        columns (list of str): List of column names to check.

    Returns:
        dict: A dictionary with column names as keys and their unique value if single, else None.
    """
    unique_values = {}
    for col in columns:
        unique_vals = df[col].unique()
        if len(unique_vals) == 1:
            unique_values[col] = unique_vals[0]
        else:
            unique_values[col] = None
    return unique_values

def return_correct_flipped_correct(df, prefix=""):
    df.loc[:, f'{prefix}correct'] = (df['Origin'] == df[f'{prefix}choice']).astype(float)
    df.loc[:, f'{prefix}flipped_correct'] = (df['Origin'] == df[f'{prefix}flipped_choice']).astype(float)
    df.loc[:, f'{prefix}avg_correct'] = (df[f'{prefix}correct'] + df[f'{prefix}flipped_correct']) / 2
    df.loc[:, f'{prefix}real_correct'] = ((df['Origin'] == df[f'{prefix}choice']) & (df['Origin'] == df[f'{prefix}flipped_choice'])).astype(int)
    return df[f'{prefix}correct'].mean(), df[f'{prefix}flipped_correct'].mean(),df[f'{prefix}avg_correct']. mean(), df[f'{prefix}real_correct'].sum()

def get_attacks(prefix, distinguisher):
    combined_dataframes = combine_files_by_base_name(prefix, distinguisher)
    all_attacks = []
    all_num_attacks = []
    for base_name, combined_df in combined_dataframes.items():
        combined_df['domain'] = base_name.split('_')[-1]  # Infer domain from base name
        attacks = separate_attacks(combined_df)
        # attacks = add_entropy_column(attacks)

        num_attacks = [attack[attack['Num'] >= 100] for attack in attacks]

        all_attacks.extend(attacks)
        all_num_attacks.extend(num_attacks)
    return all_attacks, all_num_attacks


In [ ]:
from collections import defaultdict
def print_stats(prefix, distinguisher):
    all_attacks, all_num_attacks = get_attacks(prefix, distinguisher)
    counter = defaultdict(int)
    countertail = defaultdict(int)
    for attack in all_attacks:
        return_correct_flipped_correct(attack)
        for i in range(len(attack)):
            counter[attack['avg_correct'].values[i]] += 1
        countertail[attack['avg_correct'].values[-1]] += 1
    score = sum(k * v for k, v in counter.items()) / sum(counter.values())
    print(prefix, distinguisher, score, list(counter.items()))

# prefix = "llama3.1-8B"
# for distinguisher in ["SimpleDistinguisher", "SimpleGPT", "SimplestGPT", "AggressiveSimple", "LogicGPT", "LogicGPTrev", "LogicSimple", "LogicSimplest"]:
#     print_stats(prefix, distinguisher)
# prefix = "gpt"
# for distinguisher in ["SimplestGPT"]:
#     print_stats(prefix, distinguisher)
# prefix = "llama3.1-70B"
# for distinguisher in ["SimpleDistinguisher", "AggressiveSimple", "LogicGPT", "LogicSimplest", "SimplestGPT"]:
#     print_stats(prefix, distinguisher)
prefix = "llama3.1-70B-full"
for distinguisher in ["SimpleDistinguisher"]:
    print_stats(prefix, distinguisher)
prefix = "llama3.1-70B-long"
for distinguisher in ["SimpleDistinguisher"]:
    print_stats(prefix, distinguisher)

llama3.1-70B-full SimpleDistinguisher 0.9986589698046181 [(1.0, 56149), (0.5, 151)]
llama3.1-70B-long SimpleDistinguisher 0.9379973474801061 [(1.0, 1321), (0.5, 187)]


In [ ]:
prefix = "llama3.1-70B-full"
distinguisher = "SimpleDistinguisher"
all_attacks, all_num_attacks = get_attacks(prefix, distinguisher)

# Combine original DataFrames for concatenated view (optional)
long = pd.concat(all_attacks, ignore_index=True)
len(long)

56300

In [ ]:
data = []
failed = []

for attack in all_attacks:    
    correct, flipped_correct, avg_correct, real_correct = return_correct_flipped_correct(attack)
    d = check_unique_values(attack, ['domain', 'entropy', 'w_str', 'm_str'])

    data.append({
        'domain' : d['domain'],
        'entropy' : d['entropy'],
        'w_str': d['w_str'],
        'm_str': d['m_str'],
        'correct': real_correct,
        'avg_correct': avg_correct,
        'total': len(attack),
        'correct_groups': real_correct == len(attack),
        'groups': 1
    })
    if real_correct != len(attack):
        failed.append({
            'domain' : d['domain'],
            'entropy' : d['entropy'],
            'w_str': d['w_str'],
            'm_str': d['m_str'],
            'correct': real_correct,
            'avg_correct': avg_correct,
            'total': len(attack),
            'correct_groups': real_correct == len(attack),
            'groups': 1
        })
    
data_df = pd.DataFrame(data)
failed_df = pd.DataFrame(failed)

In [ ]:
def group_df1(df, group_col, stat):
    grouped_df = (
        df.groupby(group_col)
          .agg({
            'avg_correct': stat,
          })
          .reset_index()
    )
    return grouped_df

def group_df2(df, group_col, stat):
    grouped_df = (
        df.groupby(group_col)
          .agg({
            # 'correct': stat,
            # 'total': stat,
            # 'correct_groups': stat,
            'groups': stat,
          })
          .reset_index()
    )
    return grouped_df

show(group_df2(data_df, ['m_str'], 'sum'))
show(group_df2(failed_df, ['m_str'], 'sum'))

m_str,groups
Document1StepMutator,576
Document2StepMutator,678
DocumentMutator,276
EntropyWordMutator,720
SentenceMutator,720
SpanMutator,720
WordMutator,720


m_str,groups
Document2StepMutator,1
DocumentMutator,2
SentenceMutator,38
SpanMutator,12


In [ ]:
# Create a list to store the attacks and their corresponding avg values
attack_avg_list = []

for i, attack in enumerate(all_attacks):
    attack['id'] = i
    correct, flipped, avg, _ = return_correct_flipped_correct(attack)
    attack_avg_list.append((attack, avg))  # Append the attack and its avg value as a tuple

# Sort the list by the avg value (second element of the tuple)
sorted_attacks = sorted(attack_avg_list, key=lambda x: x[1])

# Extract the sorted attacks and their averages separately if needed
sorted_attacks_list = [item[0] for item in sorted_attacks]
sorted_avgs = [item[1] for item in sorted_attacks]

# # Optionally, print the sorted averages for verification
# for attack, avg in sorted_attacks:
#     print(f"Attack: {attack}, Avg: {avg}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, IntSlider

# Function to plot cumulative avg progression for a given attack index
def plot_cumulative_avg(attack_index):
    # Get the attack for the given index
    attack = sorted_attacks_list[attack_index]

    avg_values = []
    
    # Iterate over the rows of the DataFrame cumulatively
    for i in range(1, len(attack) + 1):
        # Select the first `i` rows
        subset = attack.iloc[:i]
        
        # Apply `return_correct_flipped_correct` to the subset
        correct, flipped, avg, _ = return_correct_flipped_correct(subset)
        
        # Store the avg value
        avg_values.append(avg)

    # Plot the graph
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(avg_values) + 1), avg_values, marker='o')
    plt.suptitle(f'Cumulative Avg Progression for Attack Index {attack_index}')
    plt.title(f'id={attack["id"].iloc[0]}, domain={attack["domain"].iloc[0]}, entropy={attack["entropy"].iloc[0]}, m_str={attack["m_str"].iloc[0]}, w_str={attack["w_str"].iloc[0]}')
    plt.xlabel('Number of Rows')
    plt.ylabel('Average')
    plt.grid(True)
    plt.show()

# Function to plot rolling avg progression for a given attack index
def plot_rolling_avg(attack_index, window_size=10):
    # Get the attack for the given index
    attack = sorted_attacks_list[attack_index]

    # Calculate rolling average
    rolling_avg = attack['avg_correct'].rolling(window=window_size).mean()

    # Plot the graph
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(rolling_avg) + 1), rolling_avg, marker='o')
    plt.suptitle(f'Rolling Avg Progression for Attack Index {attack_index} (Window Size: {window_size})')
    plt.title(f'id={attack["id"].iloc[0]}, domain={attack["domain"].iloc[0]}, entropy={attack["entropy"].iloc[0]}, m_str={attack["m_str"].iloc[0]}, w_str={attack["w_str"].iloc[0]}')
    plt.xlabel('Number of Rows')
    plt.ylabel('Rolling Average')
    plt.grid(True)
    plt.show()


# Create an interactive slider to select attack index
def plot_selected(attack_index, plot_type, window_size=10):
    if plot_type == 'cumulative':
        plot_cumulative_avg(attack_index)
    elif plot_type == 'rolling':
        plot_rolling_avg(attack_index, window_size)

interact(
    plot_selected,
    attack_index=IntSlider(min=0, max=len(sorted_attacks_list) - 1, step=1, value=100),
    plot_type={
        'Cumulative Average': 'cumulative',
        'Rolling Average': 'rolling'
    },
    window_size=IntSlider(min=1, max=50, step=1, value=10)
)

interactive(children=(IntSlider(value=100, description='attack_index', max=4409), Dropdown(description='plot_t…

<function __main__.plot_selected(attack_index, plot_type, window_size=10)>

In [ ]:
sorted_attacks_list[0][['Origin', 'choice', 'flipped_choice', 'correct', 'flipped_correct', 'avg_correct']].tail(5)

,Origin,choice,flipped_choice,correct,flipped_correct,avg_correct
175,A,A,B,1.0,0.0,0.5
176,A,A,B,1.0,0.0,0.5
177,A,A,B,1.0,0.0,0.5
178,A,A,B,1.0,0.0,0.5
179,A,A,B,1.0,0.0,0.5


In [ ]:
show(sorted_attacks_list[0][['Origin', 'prompt', 'P', 'origin_A', 'origin_B']].tail(2))

In [ ]:
# convert the list of attacks to a single dataframe
first_rows = [attack.iloc[0].to_frame().T for attack in sorted_attacks_list]
attacks_df = pd.concat(first_rows, ignore_index=True)
# add sorted_avgs as a column to the dataframe
attacks_df['avg_correct'] = sorted_avgs
attacks_df[['domain', 'entropy', 'w_str', 'm_str', 'avg_correct']][:20]

,domain,entropy,w_str,m_str,avg_correct
0,space,4,SIR,SentenceMutator,0.700000
1,space,4,KGW,SentenceMutator,0.708333
2,space,1,SIR,SentenceMutator,0.750000
3,space,2,Adaptive,SentenceMutator,0.750000
4,space,2,Adaptive,SentenceMutator,0.750000
5,space,10,SIR,SentenceMutator,0.777778
6,space,9,SIR,SentenceMutator,0.791667
7,space,9,SIR,SentenceMutator,0.791667
8,paris,5,SIR,SpanMutator,0.800000
9,space,7,Adaptive,SentenceMutator,0.807692


In [ ]:
i = 0
origin_A = sorted_attacks_list[i]['origin_A'].values[0]
origin_B = sorted_attacks_list[i]['origin_B'].values[0]
P = sorted_attacks_list[i]['P'].values[-1]

print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_A}
___________________________________________________________
Original response B: {origin_B}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: Venturing into space is a groundbreaking endeavor that unlocks a multitude of benefits, extending far beyond the realms of scientific discovery and territorial growth. Space exploration, frequently overlooked, is a catalyst for scientific progress, driving the development of pioneering technologies and addressing humanity's most pressing challenges directly, making it a pursuit of paramount importance that warrants greater acknowledgment and support. This essay examines the importance of space exploration, its potential to broaden our understanding, and its ability to contribute to resolving critical global challenges like environmental decay and resource exhaustion. Understanding the cosmos is vital, as it allows us to grasp the intricate mechanisms governing the universe and our place within it, ultimately expanding our comprehension of reality itself. Exploring the vastness of the universe reveals a profound comprehension of the fundamental laws that shape reality, th

In [ ]:
print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_B}
___________________________________________________________
Original response B: {origin_A}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: Venturing into space is a groundbreaking endeavor that unlocks a multitude of benefits, extending far beyond the realms of scientific discovery and territorial growth. Space exploration, frequently overlooked, is a catalyst for scientific progress, driving the development of pioneering technologies and addressing humanity's most pressing challenges directly, making it a pursuit of paramount importance that warrants greater acknowledgment and support. This essay examines the importance of space exploration, its potential to broaden our understanding, and its ability to contribute to resolving critical global challenges like environmental decay and resource exhaustion. Understanding the cosmos is vital, as it allows us to grasp the intricate mechanisms governing the universe and our place within it, ultimately expanding our comprehension of reality itself. Exploring the vastness of the universe reveals a profound comprehension of the fundamental laws that shape reality, th

In [ ]:
show(sorted_attacks_list[0].tail(1))